# Assignment 4 - Questions
## Q1
### a)
The main difference between the 'convolutions' in CNNs and the convolutions in other computer vision algorithms is through the construction of the kernel filters. In most computer vision algorithms, the kernel is constructed by the user in order to perform a specific task (e.g. Harris Corner Detector to find corners in the image). On the other hand, CNNs aim to learn the weights of the kernel filter, where the functionality of the filter is not necessarily known to an outside observer.
### b)
The main advantage of user-constructed filters is its algorithmic applicability. That is to say, because the filter and its functionality is known, different kernels can be selectively chosen to perform specific tasks depending on the algorithm. In the SIFT algorithm, kernel filters are used to extract corners in an image, which are used to represent fundamental features of the image. However, in many cases, human-defined characterizations of what features are fundamental to an image tend to not be effective. This is the main advantage of CNNs. By simply defining a loss and an architecture, we allow a network to self-learn the fundamental features of an image, leading to locally optimal results for the given architecture. Of course, the main disadvantage of this approach is the lack of understandability for a human interpreter, thus making neural network approaches 'black box' approaches.
## Q2
A locally-connected MLP likely will have worse results compared to one that's densely connected. Each perceptron in a locally connected MLP only takes in as input the outputs of the perceptrons closest to it in the previous layer, in contrast to a densely connected layer which takes all previous outputs. Although a locally connected MLP may require fewer operations, resulting in a faster network, it is likely that just the locally connected features are insufficient for making a classification. In a famous example, a group of blind men who have never come across an elephant try to imagine what an elephant is like by touching it. However, they can only touch one body part. Unsurprisingly, each man came to a different conclusion about the elephant depending on the body part: the man who touched the trunk compared it to a snake, the man who touched the leg compared it to a tree, etc. We can think of each of these blind men as the learned convolutional feature maps in a CNN, where each convolutional filter learns a different feature of the image. A densely connected layer is important for taking the global context from every convolutional filter in order to make an observation.
## Q3
Learning rate, batch size, and training time are all important hyperparameters to select when training a neural network. Learning rate denotes the step size for the algorithm to use when moving along the gradient towards to global minimum. A large step size may result in the network overstepping the global minimum and never being able to find the true optimum. On the other hand, a small step size is much less likely to overshoot the global/local minimum but will result in a much slower training speed, since more steps will be required to converge. Batch size denotes the subset of the training data used to train the model during each epoch of training. Generally large batch sizes are preferred since they allow faster computations through GPU parallelization, although this takes a lot of memory. Using a larger batch size also guarantees convergence to the global optima of the objective function. However, it is found empirically that too large of a batch size leads to slower convergence to the optima as well as poorer generalization of the network. On the other hand, smaller batch sizes allow for faster convergence to a local optima and good generalization, at the cost of likely not converging to the global maxima of the objective function. Lastly, training time is used to denote the number of epochs that the network trains on the dataset. It is necessary to have training time long enough such that the network may learn the dataset, however too long training times will result in overfitting, or memorization of the data, and thus poor generalization.
## Q4
Effect of $1\times 1\times d$ max pooling layer
- Increases computational cost of training                  F
- Decreases computational cost of training                  T
- Increases computational cost of testing                   F
- Decreases computational cost of testing                   T
- Increases overfitting                                     F
- Decreases overfitting                                     T
- Increases underfitting                                    T
- Decreases underfitting                                    F
- Increases the nonlinearity of the decision function       T
- Decreases the nonlinearity of the decision function       F
- Provides local rotational invariance                      F
- Provides global rotational invariance                     T
- Provides local scale invariance                           F
- Provides global scale invariance                          T
- Provides local translational invariance                   F
- Provides global translational invariance                  T

## Q5
The single layer perceptron with 10 neurons is implemented as follows:

In [ ]:
import numpy as np
import random
from code import hyperparameters as hp

def train_nn(self):
     # This is our training data as indices into an image storage array
    indices = list(range(self.train_images.shape[0]))

    # These are our storage variables for our update gradients.
    # delta_W is the matrix of gradients on the weights of our neural network
    #     Each row is a different neuron (with its own weights)
    # delta_b is the vector of gradients on the biases (one per neuron)
    delta_W = np.zeros((self.input_size, self.num_classes))
    delta_b = np.zeros((1, self.num_classes))

    # Iterate over the number of epochs declared in the hyperparameters.py file
    for epoch in range(hp.num_epochs):
        # Overall per-epoch sum of the loss
        loss_sum = 0
        # Shuffle the data before training each epoch to remove ordering bias
        random.shuffle(indices)

        # For each image in the datset:
        for index in range(len(indices)):
            # Get input training image and ground truth label
            i = indices[index]
            img = self.train_images[i]
            gt_label = self.train_labels[i]

            ################
            # GENERAL ADVICE:
            # This is _precise work_ - we need very few lines of code.
            # At this point, we need not write any for loops.
            # As a guide, the solution for this can be writtein in 14 lines of code.
            #
            # Further, please do not use any library functions.

            ################
            # FORWARD PASS:
            # This is where we take our current estimate of the weights and biases
            # and compute the current error against the training data.

            # Step 1:
            # Compute the output response to this 'img' input for each neuron (linear unit).
            # Our current estimate for the weights and biases are stored in:
            #    self.W
            #    self.b
            # Remember: use matrix operations.
            # Our output will be a number for each neuron stored in a vector.
            out = np.matmul(img, self.W) + self.b

            # Step 2:
            # Convert these to probabilities by implementing the softmax function.
            prob = np.exp(out) / (np.sum(np.exp(out)) + 1E-10)

            # Step 3:
            # Compute the error against the training label 'gt_label' using the cross-entropy loss
            # Remember:
            #     log has a potential divide by zero error
            y = np.zeros(self.num_classes)
            y[gt_label] = 1
            loss_over_all_classes = - np.log(prob + 1E-10)
            loss_sum = loss_sum + loss_over_all_classes[:, gt_label]


            ################
            # BACKWARD PASS (BACK PROPAGATION):
            # This is where we find which direction to move in for gradient descent to
            # optimize our weights and biases.
            # Use the derivations from the questions handout.

            # Step 4:
            # Compute the delta_W and delta_b gradient terms for the weights and biases
            # using the provided derivations in Eqs. 6 and 7 of the handout.
            # Remember:
            #    delta_W is a matrix the size of the input by the size of the classes
            #    delta_b is a vector
            # Note:
            #    By equation 6 and 7, we need to subtract 1 from p_j only if it is
            #    the true class probability.
            delta_b = prob - y
            delta_W = np.outer(img, delta_b)


            # Step 5:
            # Update self.W and self.b using the gradient terms
            # and the self.learning_rate hyperparameter.
            # Eqs. 4 and 5 in the handout.
            self.b = self.b - self.learning_rate * delta_b
            self.W = self.W - self.learning_rate * delta_W